In [ ]:
#versions: pandas==2.0.3, numpy==1.24.4

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype


In [ ]:
FASTSURFER_OUTPUT_DIR="./data/fastsurfer_output/"
OASIS_DATATABLES_DIR="./data/OASIS/OASIS_Meta/OASIS_Data/"
MAPPING_FILE="./data/texture_mapping.csv"
OASIS_TEXTURE_DATA="./data/OASIS/summary_scan_level.csv"
FREESURFER_LUT_FILE="./data/Freesurfer-LUT_new.txt"
OASIS_SAVE_NAME="./data/OASIS_all.csv"

In [ ]:
data_aseg=pd.read_table(FASTSURFER_OUTPUT_DIR+"aseg_stats.txt")
data_wmparc=pd.read_table(FASTSURFER_OUTPUT_DIR+"wmparc_stats.txt")
data_lhDKT=pd.read_table(FASTSURFER_OUTPUT_DIR+"lh.aparc.DKTatlas.mapped.volume.txt")
data_rhDKT=pd.read_table(FASTSURFER_OUTPUT_DIR+"rh.aparc.DKTatlas.mapped.volume.txt")

data_lhDKT=data_lhDKT.rename({"lh.aparc.DKTatlas.mapped.volume":"filename"},axis=1)
data_rhDKT=data_rhDKT.rename({"rh.aparc.DKTatlas.mapped.volume":"filename"},axis=1)
data_aseg=data_aseg.rename({"Measure:volume":"filename"},axis=1)
data_wmparc=data_wmparc.rename({"Measure:volume":"filename"},axis=1)

df=pd.merge(data_lhDKT,data_rhDKT,on="filename",suffixes=("", "_y"))

df=pd.merge(df,data_wmparc,on="filename",suffixes=("", "_y"))
df=pd.merge(df,data_aseg,on="filename",suffixes=("", "_y"))


cols = [c for c in df.columns if not c.endswith("_y")]
df=df[cols]

df["PTID"]=df.filename.str.split("_").str[0].str[4:]

df.iloc[:,1:191]=df.iloc[:,1:191].div(df.EstimatedTotalIntraCranialVol, axis=0)

In [ ]:
dataMRI=pd.read_csv(OASIS_DATATABLES_DIR+"MRT_Meta.csv")

dataMRI["daysFromBL"]=dataMRI["MR ID"].str[13:]
dataMRI["daysFromBL"]=dataMRI["daysFromBL"].astype("int32")

In [ ]:
dataMRIRes=df

In [ ]:
dataMRIRes["daysFromBL"]=dataMRIRes.filename.str[18:22]
dataMRIRes["daysFromBL"]=dataMRIRes["daysFromBL"].astype("int32")

In [ ]:
dataMRITest=pd.merge(dataMRI,dataMRIRes,left_on=["Subject","daysFromBL"],right_on=["PTID","daysFromBL"])
dataMRITest=dataMRITest.drop_duplicates(["PTID","daysFromBL"])
dataMRITest=dataMRITest.drop(["Date","Subject","Scans","FreeSurfers","PUPs"],axis=1)

In [ ]:
dataDiag=pd.read_csv(OASIS_DATATABLES_DIR+"D1.csv")

dataDiag=dataDiag[~dataDiag.DLB.isin([1])&~dataDiag.DLBIF.isin([1,2])&~dataDiag.VASC.isin([1])&~dataDiag.VASCIF.isin([1,2])&~dataDiag.VASCPS.isin([1])&~dataDiag.VASCPSIF.isin([1,2])&~dataDiag.STROKE.isin([1])&~dataDiag.STROKIF.isin([1,2])&~dataDiag.ALCDEM.isin([1])&~dataDiag.ALCDEMIF.isin([1,2])&~dataDiag.FTD.isin([1])&~dataDiag.FTDIF.isin([1,2])&~dataDiag.PPAPH.isin([1])&~dataDiag.PPAPHIF.isin([1,2])&~dataDiag.PSP.isin([1])&~dataDiag.PSPIF.isin([1,2])&~dataDiag.CORT.isin([1])&~dataDiag.CORTIF.isin([1,2])&~dataDiag.HUNT.isin([1])&~dataDiag.HUNTIF.isin([1,2])&~dataDiag.PRION.isin([1])&~dataDiag.PRIONIF.isin([1,2])&~dataDiag.MEDS.isin([1])&~dataDiag.MEDSIF.isin([1,2])&~dataDiag.DYSILL.isin([1])&~dataDiag.DYSILLIF.isin([1,2])&~dataDiag.DEP.isin([1])&~dataDiag.DEPIF.isin([1,2])&~dataDiag.DOWNS.isin([1])&~dataDiag.DOWNSIF.isin([1,2])&~dataDiag.OTHPSYIF.isin([1,2])&~dataDiag.PARK.isin([1])&~dataDiag.PARKIF.isin([1,2])&~dataDiag.HYCEPH.isin([1])&~dataDiag.HYCEPHIF.isin([1,2])&~dataDiag.BRNINJ.isin([1])&~dataDiag.BRNINJIF.isin([1,2])&~dataDiag.NEOP.isin([1])&~dataDiag.NEOPIF.isin([1,2])&~dataDiag.COGOTH.isin([1])&~dataDiag.COGOTHIF.isin([1,2])&~dataDiag.COGOTH2.isin([1])&~dataDiag.COGOTH2F.isin([1,2])&~dataDiag.COGOTH3.isin([1])&~dataDiag.COGOTH3F.isin([1,2])]

dataDiag["UDS_D1DXDATA ID"].str[-4:]

dataDiag["daysFromBL"]=dataDiag["UDS_D1DXDATA ID"].str[-4:]
dataDiag["daysFromBL"]=dataDiag["daysFromBL"].astype("int32")

dataDiag=dataDiag.filter(["Subject","MCIAMEM","MCIAPLUS","PROBADIF","PROBAD","POSSADIF","POSSAD","NORMCOG","UDS_D1DXDATA ID","daysFromBL"],axis=1)

MCI=dataDiag[((~(pd.isna(dataDiag.MCIAMEM)))|(~(pd.isna(dataDiag.MCIAPLUS))))]

MCI=MCI[((MCI.MCIAMEM==1)|(MCI.MCIAPLUS==1))]

AD=dataDiag[(dataDiag.PROBADIF==1)|(dataDiag.POSSADIF==1)]

CN=dataDiag[(~pd.isna(dataDiag.NORMCOG))]
CN=CN[(CN.NORMCOG==1)]

AD_red=AD[~(AD["UDS_D1DXDATA ID"].isin(CN["UDS_D1DXDATA ID"]))&~(AD["UDS_D1DXDATA ID"].isin(MCI["UDS_D1DXDATA ID"]))]

MCI_red=MCI[~(MCI["UDS_D1DXDATA ID"].isin(CN["UDS_D1DXDATA ID"]))&~(MCI["UDS_D1DXDATA ID"].isin(AD["UDS_D1DXDATA ID"]))]
CN_red=CN[~(CN["UDS_D1DXDATA ID"].isin(MCI["UDS_D1DXDATA ID"]))&~(CN["UDS_D1DXDATA ID"].isin(AD["UDS_D1DXDATA ID"]))]

AD_data=pd.DataFrame({"ID":AD_red["UDS_D1DXDATA ID"],"Subject":AD_red.Subject,"daysFromBL":AD_red.daysFromBL})
AD_data["DX"]="Dementia"



CN_data=pd.DataFrame({"ID":CN_red["UDS_D1DXDATA ID"],"Subject":CN_red.Subject,"daysFromBL":CN_red.daysFromBL})
CN_data["DX"]="CN"

MCI_data=pd.DataFrame({"ID":MCI_red["UDS_D1DXDATA ID"],"Subject":MCI_red.Subject,"daysFromBL":MCI_red.daysFromBL})
MCI_data["DX"]="MCI"

dataDiag=pd.concat([AD_data,CN_data,MCI_data])

In [ ]:
dataDemo=pd.read_csv(OASIS_DATATABLES_DIR+"Demography.csv")
dataDemo=dataDemo.drop_duplicates("Subject")
dataDemo=dataDemo.filter(["Subject","SEX"],axis=1)

dataDemo.columns=["Subject","PTGENDER"]

dataDemo.PTGENDER[dataDemo.PTGENDER==1]="Male"
dataDemo.PTGENDER[dataDemo.PTGENDER==2]="Female"
dataDemo.PTGENDER[dataDemo.PTGENDER==0]=np.nan


In [ ]:
dataClin=pd.read_csv(OASIS_DATATABLES_DIR+"Clinical_Data.csv")


dataClin["daysFromBL"]=dataClin["ADRC_ADRCCLINICALDATA ID"].str.split("_d").str[-1]
dataClin["daysFromBL"]=dataClin["daysFromBL"].astype("int32")

dataClin=dataClin.filter(["Subject","daysFromBL","apoe","mmse","cdr","ageAtEntry"],axis=1)

dataClin.columns=["Subject","daysFromBL","APOE4","MMSCORE","CDGLOBAL","ageAtEntry"]

dataClin.APOE4[dataClin.APOE4==22]=0
dataClin.APOE4[dataClin.APOE4==23]=0
dataClin.APOE4[dataClin.APOE4==24]=1
dataClin.APOE4[dataClin.APOE4==33]=0
dataClin.APOE4[dataClin.APOE4==34]=1
dataClin.APOE4[dataClin.APOE4==44]=2

dataDiag=pd.merge(dataDiag,dataDemo,on=["Subject"])

dataDiag=pd.merge(dataDiag,dataClin,on=["Subject","daysFromBL"],how="left")

In [ ]:
dataTests=pd.read_csv(OASIS_DATATABLES_DIR+"Psychometrics.csv")

dataTests["daysFromBL"]=dataTests["CNDA_PSYCHOMETRICSDATA ID"].str[-4:]

dataTests["daysFromBL"]=dataTests["daysFromBL"].astype("int32")

dataTests=dataTests.filter(["Subject","daysFromBL","LOGIMEM","MEMUNITS"],axis=1)

dataTests.columns=["Subject","daysFromBL","LIMMTOTAL","LDELTOTAL"]

dataU=pd.merge(dataDiag,dataTests,on="Subject",how="left")

dataU["dist"]=abs(dataU["daysFromBL_x"]-dataU["daysFromBL_y"])

dataU.dist[pd.isnull(dataU.dist)] = 365.25

dataUTest=dataU.loc[dataU.reset_index().groupby("ID").dist.idxmin()].reset_index(drop=True)

dataUTest.LIMMTOTAL[dataUTest.dist>365.25] =np.nan
dataUTest.LDELTOTAL[dataUTest.dist>365.25] =np.nan
dataUTest.dist[dataUTest.dist>365.25] =365.25

dataUTest=dataUTest.drop(["ID","daysFromBL_y","dist"],axis=1)


dataUTest=dataUTest.rename({"daysFromBL_x":"daysFromBL"},axis=1)

In [ ]:
dataMRITest=dataMRITest.rename({"PTID":"Subject"},axis=1)

dataGes=pd.merge(dataMRITest,dataUTest,on="Subject")

dataGes["dist"]=abs(dataGes.daysFromBL_x-dataGes.daysFromBL_y)

gender_mass1=dataGes.loc[dataGes.reset_index().groupby("MR ID").dist.idxmin()].reset_index(drop=True)

gender_mass1Filtered=gender_mass1[gender_mass1.dist<=365.25]

gender_mass1Filtered=gender_mass1Filtered.reset_index(drop=True)


gender_mass1Filtered=gender_mass1Filtered[~gender_mass1Filtered.filename.isin(["sub-OAS30331_ses-d2229_run-02_T1w-flirt/","sub-OAS30420_ses-d2792_run-01_T1w-flirt/","sub-OAS30505_ses-d4682_run-01_T1w-flirt/","sub-OAS31100_ses-d7350_run-01_T1w-flirt/","sub-OAS31012_ses-d0308_run-01_T1w-flirt/","sub-OAS30704_ses-d0584_run-01_T1w-flirt/"])]

merged_OASIS=gender_mass1Filtered
merged_OASIS=merged_OASIS.drop(["daysFromBL_y"],axis=1)


merged_OASIS=merged_OASIS.rename({"daysFromBL_x":"daysFromBL"},axis=1)
merged_OASIS.index=merged_OASIS.Subject

merged_OASIS=merged_OASIS.drop(["Subject"],axis=1)

In [ ]:
oasis_DXbl=pd.DataFrame(columns=["PTID","DX_bl"])
for subj in merged_OASIS.index.unique():
    oasis_PTID=dataDiag[dataDiag["Subject"]==subj]
    oasis_PTID=oasis_PTID.sort_values("daysFromBL")
    DXBL=oasis_PTID.iloc[0]["DX"]
    oasis_DXbl=pd.concat([oasis_DXbl,pd.DataFrame({"PTID":[subj],"DX_bl":[DXBL]})])

oasis_DXbl=oasis_DXbl[oasis_DXbl.DX_bl.isin(["CN","MCI"])]

oasis_DX_test=pd.merge(oasis_DXbl,dataDiag,left_on="PTID",right_on="Subject")

In [ ]:

cat_type = CategoricalDtype(categories=["CN", "MCI", "Dementia"], ordered=True)

oasis_DX_test["DX"] = oasis_DX_test["DX"].astype(cat_type)

In [ ]:
df_tte=pd.DataFrame(columns=["PTID","Status","Time","Ref_age"])
for PTID in oasis_DX_test.PTID.unique():
    df_PTID=oasis_DX_test[oasis_DX_test.PTID==PTID]
    df_PTID=df_PTID.sort_values("daysFromBL")
    status=""
    time=0.0
    if len(df_PTID.DX.unique())==1:
        if (df_PTID.DX.unique()[0]=="MCI"):
            status="sMCI"
            time=df_PTID.daysFromBL.max()-df_PTID.daysFromBL.min()
            ref_age=df_PTID.daysFromBL.min()
        else:
            if (df_PTID.DX.unique()[0]=="CN"):
                status="sCN"
                time=df_PTID.daysFromBL.max()-df_PTID.daysFromBL.min()
                ref_age=df_PTID.daysFromBL.min()
    else:
        if "Dementia" in df_PTID.DX.tolist():
            if (all((df_PTID.sort_values("DX").reset_index().daysFromBL)==(df_PTID.reset_index().daysFromBL))):
                if df_PTID.iloc[0].DX=="MCI":
                    status="pMCI"
                else:
                    if df_PTID.iloc[0].DX=="CN":
                        status="pCN"
                df_PTID_AD=df_PTID[df_PTID.DX=="Dementia"]
                time=df_PTID_AD.daysFromBL.min() - df_PTID.daysFromBL.min()
                ref_age=df_PTID.daysFromBL.min()
            else:
                if df_PTID.iloc[0].DX=="MCI":
                    df_PTID_AD=df_PTID[df_PTID.DX=="Dementia"]
                    df_PTID_CN=df_PTID[df_PTID.DX=="CN"]
                    df_PTID_MCI=df_PTID[df_PTID.DX=="MCI"]
                    value_AD=df_PTID_AD.daysFromBL.min()
                    value_MCI=df_PTID_MCI.daysFromBL.max()
                    if(df_PTID_CN.shape[0]==0):
                        value_CN=0
                    else:
                        value_CN=df_PTID_CN.daysFromBL.max()
                    if(value_CN<value_AD and value_MCI<value_AD):
                        status="uMCIADClear"
                    else:
                        status="uMCIADUnclear"
                else:
                    if df_PTID.iloc[0].DX=="CN":
                        df_PTID_AD=df_PTID[df_PTID.DX=="Dementia"]
                        df_PTID_CN=df_PTID[df_PTID.DX=="CN"]
                        df_PTID_MCI=df_PTID[df_PTID.DX=="MCI"]
                        value_AD=df_PTID_AD.daysFromBL.min()
                        value_CN=df_PTID_CN.daysFromBL.max()
                        if(df_PTID_MCI.shape[0]==0):
                            value_MCI=0
                        else:
                            value_MCI=df_PTID_MCI.daysFromBL.max()
                        if(value_CN<value_AD and value_MCI<value_AD):
                            status="uCNADClear"
                        else:
                            status="uCNADUnclear"
                df_PTID_AD=df_PTID[df_PTID.DX=="Dementia"]
                time=df_PTID_AD.daysFromBL.min()-df_PTID.daysFromBL.min()
                ref_age=df_PTID.daysFromBL.min()
        else:
            if (all((df_PTID.sort_values("DX").reset_index().daysFromBL)==(df_PTID.reset_index().daysFromBL))):
                if df_PTID.iloc[0].DX=="CN":
                    status="CNtoMCI"
                    time=df_PTID.daysFromBL.max()-df_PTID.daysFromBL.min()
                    ref_age=df_PTID.daysFromBL.min()
            else:
                if df_PTID.iloc[0].DX=="MCI":
                    status="uMCINoAD"
                else:
                    if df_PTID.iloc[0].DX=="CN":
                        status="uCNNoAD"
                time=df_PTID.daysFromBL.max()-df_PTID.daysFromBL.min()
                ref_age=df_PTID.daysFromBL.min()
    if not ((len(df_PTID.DX.unique())==1) and (df_PTID.iloc[0].DX=="Dementia")):
        df_tte=pd.concat([df_tte,pd.DataFrame({"PTID":[PTID],"Status":[status], "Time": [time],"Ref_age":[ref_age]})],ignore_index=True)


In [ ]:
df_tte=df_tte[df_tte.Time>0]

df_tte=df_tte[df_tte.Status.isin(["sCN","sMCI","pMCI","uMCINoAD","uCNNoAD","uMCIADClear","uCNADClear","CNtoMCI","pCN"])]

merged_OASIS=merged_OASIS.reset_index()

merged_OASIS=pd.merge(merged_OASIS,df_tte,left_on="Subject",right_on="PTID")

merged_OASIS=merged_OASIS[merged_OASIS.DX.isin(["CN","MCI"])]

df_ges=merged_OASIS

df_ges.Time=df_ges.Time-(df_ges.daysFromBL-df_ges.Ref_age)

df_ges.Time=df_ges.Time/365.25

df_ges_test=df_ges

df_ges=df_ges[df_ges.Time>0.1]

df_ges=df_ges.sort_values(["Age"]).drop_duplicates("Subject")

cols = [c for c in df_ges.columns if not c.endswith("_y")]
df_ges=df_ges[cols]

In [ ]:
volume_feature=pd.read_csv(MAPPING_FILE)


df_ges.loc[df_ges.Scanner=="1.5T","Scanner"]="1.5"
df_ges.loc[df_ges.Scanner=="3.0T","Scanner"]="3.0"

df_ges=df_ges.rename({"Scanner":"MagStrength","Age":"AGE"},axis=1)

df_ges=df_ges.filter(["PTID","PTGENDER","CDGLOBAL","MMSCORE","APOE4","AGE","IMAGEUID","MagStrength","Status","Time","LDELTOTAL","LIMMTOTAL","filename","EstimatedTotalIntraCranialVol"]+volume_feature.region_volume.tolist(),axis=1)

In [ ]:
texture=pd.read_csv(OASIS_TEXTURE_DATA)

texture["filename"]=texture.path.str.split("/").str[5]+"/"

texture=texture.drop(["path"],axis=1)

freeSurfer_LUT=pd.read_table(FREESURFER_LUT_FILE, sep=";")

df=pd.DataFrame([texture.drop(["filename"],axis=1).columns,texture.drop(["filename"],axis=1).columns.str.split("_").str[-1],texture.drop(["filename"],axis=1).columns.str.rsplit("_", n=1).str.get(0)]).transpose()


df.columns=["feature_name","region","feature_name_without_region"]

df.region=pd.to_numeric(df.region)
dfmerged=pd.merge(df,freeSurfer_LUT,left_on="region",right_on="0", how="left")
cnames=dfmerged.feature_name+"_"+dfmerged.Unknown
cnames=cnames.tolist()
cnames.append("filename")
texture.columns=cnames

In [ ]:
df_ges=pd.merge(df_ges,texture,on="filename",how="left",suffixes=["","_y"])
cols = [c for c in df_ges.columns if not c.endswith("_y")]
df_ges=df_ges[cols]

df_ges=df_ges.dropna(thresh=int(len(df_ges)*0.5), axis=1)

In [ ]:
list_Y_train=list()
for i, dat in df_ges.iterrows():
    statNew=False
    if (dat.Status=="sCN") or (dat.Status=="sMCI") or (dat.Status=="CNtoMCI") or (dat.Status=="uMCINoAD") or (dat.Status=="uCNNoAD"):
        statNew=False
    else:
        statNew=True
    list_Y_train.append(statNew)

df_ges["out"]=list_Y_train

In [ ]:
list_Y_train=list()
for i, dat in df_ges.iterrows():
    statNew="CN"
    if (dat.Status=="sMCI") or  (dat.Status=="pMCI")  or (dat.Status=="uMCINoAD") or (dat.Status=="uMCIADClear"):
        statNew="MCI"
    list_Y_train.append(statNew)

df_ges["init_diag"]=list_Y_train

In [ ]:
df_ges.to_csv(OASIS_SAVE_NAME)